# Recap PWD

## Simple arithmetic workflow
```json
{
  "version": "0.1.0",
  "nodes": [
    {"id": 0, "type": "function", "value": "workflow.get_prod_and_div"},
    {"id": 1, "type": "function", "value": "workflow.get_sum"},
    {"id": 2, "type": "function", "value": "workflow.get_square"},
    {"id": 3, "type": "input", "value": 1, "name": "x"},
    {"id": 4, "type": "input", "value": 2, "name": "y"},
    {"id": 5, "type": "output", "name": "result"}
  ],
  "edges": [
    {"target": 0, "targetPort": "x", "source": 3, "sourcePort": null},
    {"target": 0, "targetPort": "y", "source": 4, "sourcePort": null},
    {"target": 1, "targetPort": "x", "source": 0, "sourcePort": "prod"},
    {"target": 1, "targetPort": "y", "source": 0, "sourcePort": "div"},
    {"target": 2, "targetPort": "x", "source": 1, "sourcePort": null},
    {"target": 5, "targetPort": null, "source": 2, "sourcePort": null}
  ]
}
```

## How to represent `While`

### AST

In [2]:
import ast

code = """
x = 0
while x < 5:
    x = x + 1
"""

tree = ast.parse(code)

print(ast.dump(tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='x', ctx=Store())],
      value=Constant(value=0)),
    While(
      test=Compare(
        left=Name(id='x', ctx=Load()),
        ops=[
          Lt()],
        comparators=[
          Constant(value=5)]),
      body=[
        Assign(
          targets=[
            Name(id='x', ctx=Store())],
          value=BinOp(
            left=Name(id='x', ctx=Load()),
            op=Add(),
            right=Constant(value=1)))],
      orelse=[])],
  type_ignores=[])


### Recursion

```python
def condition(x, limit):
    return limit > x

def function_body(x):
    return x + 1

def abstract_while(x, limit):
    if not condition(x=x, limit=limit):
        return x
    x = function_body(x=x)
    return abstract_while(x=x, limit=limit)
```

- Recursive function, supported by all WfMS
- Body and condition as _inputs_ of the `"while"`, defined in Python module (compare AST)



### `flowrep` representation

- Path in `flowrep`: pure python -> AST -> custom representation:

```
{'inputs': {'x': {'default': 0}, 'limit': {'default': 5}},
 'outputs': {'x': {}},
 'nodes': {'injected_While_0': {'nodes': {'function_body_0': {'inputs': {'x': {}},
     'outputs': {'output': {}},
     'function': <function __main__.function_body(x)>,
     'type': 'Function'}},
   'edges': [('function_body_0.outputs.output', 'outputs.x'),
    ('inputs.x', 'test.inputs.x'),
    ('inputs.x', 'function_body_0.inputs.x'),
    ('inputs.limit', 'test.inputs.limit')],
   'label': 'injected_While_0',
   'inputs': {'x': {}, 'limit': {}},
   'outputs': {'x': {}},
   'test': {'inputs': {'x': {}, 'limit': {}},
    'outputs': {'output': {}},
    'function': <function __main__.condition(x, limit)>,
    'type': 'Function'}}},
 'edges': [('inputs.x', 'injected_While_0.inputs.x'),
  ('inputs.limit', 'injected_While_0.inputs.limit'),
  ('injected_While_0.outputs.x', 'outputs.x')],
 'label': 'workflow_with_while',
 'type': 'Workflow'}
```

- Additional node type `"while"` as sub-graph again with nodes and edges

## First proposal for JSON

```json
{
    "version": "0.1.1",
    "nodes": [
        {
            "id": 0, 
            "type": "while", 
            "value": {
                "nodes": [
                    {"id": 0, "type": "function", "value": "workflow.function_body"},
                    {"id": 1, "type": "test", "value": "workflow.condition"},
                    {"id": 2, "type": "input", "name": "x"},
                    {"id": 3, "type": "input", "name": "limit"},
                    {"id": 4, "type": "output", "name": "x"},
                ],
                "edges": [
                    {"target": 4, "targetPort": null, "source": 0, "sourcePort": null},
                    {"target": 0, "targetPort": "x", "source": 2, "sourcePort": null},
                    {"target": 1, "targetPort": "x", "source": 2, "sourcePort": null},
                    {"target": 1, "targetPort": "limit", "source": 3, "sourcePort": null},
                ]
            }
        },
        {"id": 1, "type": "input", "value": 0, "name": "x"},
        {"id": 2, "type": "input", "value": 5, "name": "limit"},
        {"id": 3, "type": "output", "name": "result"}
        
    ],
    "edges": [
        {"target": 0, "targetPort": "x", "source": 1, "sourcePort": null},
        {"target": 0, "targetPort": "limit", "source": 2, "sourcePort": null},
        {"target": 3, "targetPort": null, "source": 0, "sourcePort": "result"},
    ]
}
```

- Cycles in the graph, not a DAG anymore (control flow graph (CFG)) -> Problems with execution order
- Suggestion: for node `3` the output has to be `x` not `result`

## Detour: Supporting nested workflows

- Adding top-level `workflows` key and new type `workflow`
- Internal structure of each `workflow` follows v0.1.0
- Without colon, `:`, refers to a `workflows` entry in the same file
- Workflows can also be defined in separate JSON files
- Allows for nesting, grouping, defining and sharing common workflows
- Suggestion: conventional name such as `main.pwd.json` and nested workflows as e.g. `sub1.pwd.json`
- Supported by aiida, jobflow, and pyiron_workflow
- Still needs to be supported by PWD

```json
{
  "version": "0.2.0",
  "workflows": {
    "prod_div": {
      "nodes": [
        {"id": 0, "type": "function", "value": "workflow.get_prod_and_div"},
        {"id": 1, "type": "function", "value": "workflow.get_sum"},
        {"id": 2, "type": "function", "value": "workflow.get_square"},
        {"id": 3, "type": "input", "value": 1, "name": "x"},
        {"id": 4, "type": "input", "value": 2, "name": "y"},
        {"id": 5, "type": "output", "name": "result"}
      ],
      "edges": [
        {"target": 0, "targetPort": "x", "source": 3, "sourcePort": null},
        {"target": 0, "targetPort": "y", "source": 4, "sourcePort": null},
        {"target": 1, "targetPort": "x", "source": 0, "sourcePort": "prod"},
        {"target": 1, "targetPort": "y", "source": 0, "sourcePort": "div"},
        {"target": 2, "targetPort": "x", "source": 1, "sourcePort": null},
        {"target": 5, "targetPort": null, "source": 2, "sourcePort": null}
      ]
    },
    "main": {
      "nodes": [
          {"id": 0, "type": "workflow", "value": "prod_div"},  <-- defined above
          {"id": 1, "value": 1, "type": "input", "name": "a"},
          {"id": 2, "value": 2, "type": "input", "name": "b"},
          {"id": 3, "type": "workflow", "value": "example.json:main"},  <-- separate JSON
          {"id": 4, "type": "output", "name": "final_result"}
      ],
      "edges": [
          {"target": 0, "targetPort": "x", "source": 1, "sourcePort": null},
          {"target": 0, "targetPort": "y", "source": 2, "sourcePort": null},
          {"target": 3, "targetPort": "inp", "source": 0, "sourcePort": "result"},
          {"target": 4, "targetPort": null, "source": 3, "sourcePort": "out"}
      ]
    }
  }
}
```

## Avoiding cycles in the JSON graph

```python
def my_while(
    input_ports: dict,
    condition_f: Callable,
    body_f: Callable,
    finalizer: Callable
):
    ctx = {}  # Or, use `initializer_f` here
    while condition_f(input_ports, ctx):
        ctx = body_f(input_ports, ctx)
    return finalizer(input_ports, ctx) # these become output ports
```

- `while` exposes fixed inputs and outputs, defined by `input_ports` and finalizer (can be annotated?)
- Avoids cycles in the JSON graph, handling of `while` offloaded to the WfMS
- Use _Context_ (`ctx`) to track internal state
- `finalizer` to concretize and return the results from the while loop
- JSON representation is a `while` _template_, not a fully static, unrolled representation (impossible)

### Example, up for discussion

```json
{
  "version": "0.2.0",
  "workflows": {
    "main": {
      "nodes": [
          {"id": 0, "value": "filename.DOS", "type": "input:from_file", "name": "DOS"},
          {"id": 1, "value": 0.0, "type": "input", "name": "EF_left"},
          {"id": 2, "value": 10.0, "type": "input", "name": "EF_right"},
          {"id": 3, "value": 0.001, "type": "input", "name": "convergence_threshold"},
          {"id": 4, "value": 1.0, "type": "input", "name": "target_num_electrons"},
          {"id": 5, "type": "while", "value": "function:body_f", "name": "while_loop", "condition_f": "condition_f", "initializer": "initializer_f", "finalizer": "finalizer_f"},
          {"id": 6, "type": "output", "name": "EF"},
          {"id": 7, "type": "output", "name": "num_electrons"}
      ],
      "edges": [
          {"source": 0, "sourcePort": null, "target": 5, "targetPort": "dos"},
          {"source": 1, "sourcePort": null, "target": 5, "targetPort": "EF_left"},
          {"source": 2, "sourcePort": null, "target": 5, "targetPort": "EF_right"},
          {"source": 3, "sourcePort": null, "target": 5, "targetPort": "target_num_electrons"},
          {"source": 4, "sourcePort": null, "target": 5, "targetPort": "convergence_threshold"},
          {"source": 5, "sourcePort": "EF", "target": 6, "targetPort": null},
          {"source": 5, "sourcePort": "num_electrons", "target": 7, "targetPort": null}
      ]
    }
  }
}
```

```python
def initializer_f(input_ports, ctx):
    # Here you can do checks, or initialize other ctx variables.
    ctx['EF_left'] = input_ports['EF_left']
    ctx['EF_right'] = input_ports['EF_right']
    ctx['EF'] = (ctx['EF_left'] + ctx['EF_right']) / 2
    return ctx

def condition_f(input_ports, ctx):
    return (ctx['EF_right'] - ctx['EF_left']) > input_ports['convergence_threshold']

def body_f(input_ports, ctx):
    ctx['num_electrons'] = compute_num_electrons(input_ports['dos'], ctx['EF']) ### ISSUE: this should be another workflow! Could be a JSON, not a python function...
    if ctx['num_electrons'] < input_ports['target_num_electrons']:
        ctx['EF_left'] = ctx['EF']
    else:
        ctx['EF_right'] = ctx['EF']
    ctx['EF'] = (ctx['EF_left'] + ctx['EF_right']) / 2
    return ctx

def finalizer_f(input_ports, ctx):
    return {"EF": ctx['EF'], "num_electrons": ctx['num_electrons']}
```

## Open questions

- How to represent sub-workflows in the bodies of the while (json, python, mixed)?
- Port mapping?
- Agree on exact JSON format

## Todos

- Implement nesting in PWD -> pydantic models
- Implement while in PWD -> pydantic models
- Support JSON PWD -> WfMS
- Future: WfMS -> JSON PWD

## Side quests

- Upgrade PWD library to the latest aiida-workgraph and pyiron_core versions
- Integrate pyiron_workflow into PWD library for nested workflows
- Fixing bugs that appear along the way